Добрый день.
Ноутбук с готовым решением пришлось собирать быстро из огромного объёма экспериментов. Надеюсь нужную версию собрал 

1.	Для начала понял, что лучшие результаты дает линейная модель. Пробовал все бустинги, случайный лес. Линейная модель быстро и точнее, поэтому все эксперименты строи на ней
2.	Tif idf давал лучший результат чем Count
3.	4 граммы давали лучший результат, чем остальные
4.	Пред обрабатывал данные по разному, разные анализаторы пробовал, чистый текст как не странно работал лучше
5.	Пробовал долбать разные статистики типа, количество слов, количество символом, количество пунктуации, количество смайликов как абсолютном выражение так и на Единцу слова или символа. Все это не давало лучший результат чем чистый текст
6.	Далее обнаружил, что SVM работает Луше, подобрал лучшие параметры, но это очень долго, эксперименты на ней строить не возможно, поэтому далее все тестировал все равно на лог регрессии.
7.	Далее пробовал предсказать оценки на неразмеченной части датасета. Пробовал кластеризацию – ничего толкового не вышло. В итоге использовал лог регрессию для этого, объединял и далее обучал на SVM, это немного прибавило скора.
8.	Далее я понял, что пытаться предсказать оценки 2, 3 и 4 абсолютно бесполезное занятие. Есть негативный отзыв, есть позитивный, угадать поставит человек 5 или 4 не возможно, это уже очень субъективно. Поэтому я сосредоточился на бинарной классификации. Единички предсказывались очень хорошо, потери сильнее были в пятерках. Во сановном за счет иронии. Когда слова имели обратный смысл.
9.	Сделал бинарную классификацию и начал подбирать отсечку. Лучший результат дала отсечка в 0,25 в пользу положительного отзыва. То есть 0,25 

In [69]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from bayes_opt import BayesianOptimization
from importlib import reload
import catboost as cb
import lightgbm as lgb
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords
from pymystem3 import Mystem

from multiprocessing import Pool
from tqdm import tqdm
from scipy.stats import gmean, rankdata
import nltk
nltk.download('stopwords')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
import xgboost
from sklearn.metrics import roc_auc_score, f1_score,r2_score,classification_report,accuracy_score,precision_score, recall_score
from sklearn.cluster import KMeans
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix

np.random.seed(42)
random.seed(42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lqw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [359]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [360]:
df_na = df[df.grades.isna()]

In [113]:
df_na

,bank,feeds,grades,date
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,28.06.2019 13:54
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,15.07.2020 14:54
5,pochtabank,Брала кредит на стиральную машину. Все платила...,NaN,09.04.2015 17:19
6,otpbank,"Откуда взялся долг по кредитной карте, если я ...",NaN,28.01.2021 13:20
9,fk_otkritie,"Уважаемый Бинбанк, если у вас имеются какие-ли...",NaN,31.08.2018 16:48
...,...,...,...,...
74981,vtb,13 августа 2018 г. в ДО ЦИК «Каретный Ряд» под...,NaN,05.09.2018 14:09
74984,vozrozhdenie,Обман со стороны банка. При снятии наличности ...,NaN,10.02.2013 21:46
74985,v-express-bank,Здравствуйте! По действующему ипотечному креди...,NaN,23.06.2020 11:26
74988,rsb,"Впервые сталкиваюсь с такой ситуацией, когда б...",NaN,18.06.2015 13:34


In [361]:
df_ticher = df.dropna()

In [114]:
df_ticher

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,04.08.2020 6:38
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5.0,19.04.2015 20:11
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2.0,28.03.2019 18:36
...,...,...,...,...
74995,trust,Три года длится эпопея с закрытием кредитной к...,1.0,26.10.2016 11:03
74996,fk_otkritie,"Являлся клиентом Бинбанка, пока после слияния ...",2.0,01.02.2020 21:07
74997,sberbank,8 мая 2016 г. мною вносились деньги на банковс...,1.0,14.06.2016 16:03
74998,uralsib,Здравствуйте! Сегодня 21.04.2020 захожу в моби...,3.0,21.04.2020 8:24


In [362]:
df_ticher.grades.value_counts(normalize=True)

1.0    0.538872
5.0    0.276381
2.0    0.109449
3.0    0.045769
4.0    0.029528
Name: grades, dtype: float64

In [1]:
df_ticher.grades = df_ticher.grades.apply(int)

In [366]:
X_train = df_ticher.copy()

In [367]:
vec = TfidfVectorizer(ngram_range=(1, 4), max_df=1.0, min_df=100) 
bow = vec.fit_transform(X_train.feeds)
y=np.array(X_train.grades)

In [368]:
model_lr = LogisticRegression(max_iter=1000, solver= 'liblinear', penalty='l1', C=1.5, random_state=random_state)
model_lr.fit(bow, y)
pred_ = model_lr.predict_proba(vec.transform(df_na.feeds))

In [372]:
pred_na = np.argmax(pred_,axis=1)

In [377]:
pred_na = pred_na + 1

In [154]:
y_n = np.argwhere(pred_[:,3:5].sum(axis=1)>0.3)

In [156]:
len(y_n)

2888

In [158]:
y_n

array([   21,    34,    39, ..., 23510, 23511, 23514], dtype=int64)

In [159]:
df_na_target = df_na.iloc[y_n(:,0)]
df_na_target.grades.loc[:] = df_na.iloc[y_n(:,1)]

In [385]:
df_na_target.grades = df_na_target.grades.apply(int)

In [427]:
df_na_target.grades.value_counts(normalize=True)

1    0.880080
5    0.094882
2    0.018067
3    0.004123
4    0.002848
Name: grades, dtype: float64

In [388]:
len(df_na_target)

23524

In [430]:
X_train = df_ticher.copy()

In [431]:
X_train = X_train.append(df_na_target)

In [432]:
X_train

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,13.12.2016 1:05
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,04.08.2020 6:38
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,19.04.2015 20:11
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,28.03.2019 18:36
...,...,...,...,...
74981,vtb,13 августа 2018 г. в ДО ЦИК «Каретный Ряд» под...,1,05.09.2018 14:09
74984,vozrozhdenie,Обман со стороны банка. При снятии наличности ...,1,10.02.2013 21:46
74985,v-express-bank,Здравствуйте! По действующему ипотечному креди...,1,23.06.2020 11:26
74988,rsb,"Впервые сталкиваюсь с такой ситуацией, когда б...",1,18.06.2015 13:34


In [392]:
X_train.grades.value_counts(normalize=True)

1    0.645893
5    0.219453
2    0.080787
3    0.032707
4    0.021160
Name: grades, dtype: float64

In [401]:
X_train.shape

(75000, 4)

In [403]:
X_train.grades = X_train.grades.map({1:1, 2:1, 3:1, 4:5, 5:5})

In [404]:
%%time
X_tr_bin= X_train[~X_train['grades'].isin([2,3,4])]
print(X_tr_bin.shape)
y_bin = np.array(X_tr_bin.grades)
vec = TfidfVectorizer(ngram_range=(1, 4), max_df=1.0, min_df=100) 
bow = vec.fit_transform(X_tr_bin.feeds)

clf_bin = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear', random_state=random_state,cache_size=1300),\
            bootstrap=True, max_samples=0.5, n_estimators=28, n_jobs=14, random_state=random_state))
clf_bin.fit(bow, y_bin)
pred_bin = clf_bin.predict_proba(vec.transform(test.feeds))


In [ ]:
res = []
for  row in pred_bin:
    if row[1] >=0.25:
        res.append(5)
    else:
        res.append(1)

In [ ]:
sol = pd.DataFrame({'inds': test.index,
                    'grades': pred_bin})
sol.to_csv('tfidf_4gr_svm_upper_data_bin.csv', index=False)